In [11]:
import os
import mne
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import warnings
import random

warnings.filterwarnings("ignore")

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Global parameters (you can adjust these later)
T = 160
delta = 4
Delta = 8
Gamma = 30
channels = ['Oz..', 'T7..', 'Cz..']


In [12]:
def process_edf(file_path, channels, T=160, delta=4, Delta=8, Gamma=30):
    raw = mne.io.read_raw_edf(file_path, preload=True, verbose=False)
    raw.pick(channels)
    data = raw.get_data().T.astype(np.float32)
    
    # Normalize the data.
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(data).astype(np.float32)
    
    n_samples = normalized_data.shape[0]
    windows = [normalized_data[start:start+T, :] 
               for start in range(0, n_samples - T + 1, delta)]
    windows = np.array(windows, dtype=np.float32)
    
    # Create augmented samples by grouping Gamma consecutive windows with a step of Delta.
    augmented = [windows[i:i+Gamma] 
                 for i in range(0, len(windows) - Gamma + 1, Delta)]
    augmented = np.array(augmented, dtype=np.float32)
    
    return augmented


In [13]:
class PhysionetAugmentedDataset(Dataset):
    def __init__(self, subject_ids, channels, T=160, delta=4, Delta=8, Gamma=30):
        self.subject_ids = subject_ids
        self.channels = channels
        self.T = T
        self.delta = delta
        self.Delta = Delta
        self.Gamma = Gamma
        self.indices = []
        
        for label, subject in enumerate(tqdm(subject_ids, desc="Building index")):
            for r in range(1, 2):
                file_path = f'./files/S{subject:03d}/S{subject:03d}R{r:02d}.edf'
                try:
                    augmented = process_edf(file_path, channels, T, delta, Delta, Gamma)
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")
                    continue
                num_augmented = augmented.shape[0]
                for aug_idx in range(num_augmented):
                    self.indices.append((subject, r, aug_idx, file_path, label))
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        subject, r, aug_idx, file_path, label = self.indices[idx]
        augmented = process_edf(file_path, self.channels, self.T, self.delta, self.Delta, self.Gamma)
        sample = augmented[aug_idx]
        sample_tensor = torch.tensor(sample, dtype=torch.float32)
        return sample_tensor, label
    
subjects = list(range(1, 91))
dataset = PhysionetAugmentedDataset(subjects, channels, T, delta, Delta, Gamma)


Building index: 100%|██████████| 90/90 [00:03<00:00, 24.74it/s]


In [14]:
# Define subject groups:
all_indices = list(range(len(dataset)))
labels = [dataset.indices[i][-1] for i in all_indices]  # label is the last element in the index tuple

# Perform a stratified split (80% train, 20% test)
train_idx, test_idx = train_test_split(all_indices, test_size=0.2, stratify=labels, random_state=42)

# Create subset datasets using the indices.
train_dataset = Subset(dataset, train_idx)
test_dataset = Subset(dataset, test_idx)

# Create DataLoaders for both splits.
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)


In [ ]:
all_labels = []
for i in tqdm(range(len(train_dataset))):
    _, label = train_dataset[i]
    all_labels.append(label)
unique_labels, counts = np.unique(all_labels, return_counts=True)
print("Training label counts:", dict(zip(unique_labels, counts)))

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.maxpool = nn.MaxPool2d(kernel_size=(1,2))
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2,2))
        self.flatten = nn.Flatten()
        self.dense1 = nn.Linear(153600, 1024)
        self.dense2 = nn.Linear(1024, 90)
        self.dropout = nn.Dropout2d(0.5) #hyperparameter

    def forward(self, x):
        x = x.permute(0, 3, 1, 2)
        x = F.relu(self.conv1(x))
        x = self.maxpool(x)
        
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)

        x = F.relu(self.conv3(x))
        x = self.maxpool2(x)
        
        x = self.flatten(x)

        x = F.relu(self.dense1(x))
        x = self.dropout(x)
        x = self.dense2(x)
        return x
    
model = CNN().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 20  # Adjust as needed

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    all_train_preds = []
    all_train_labels = {}

    for batch_data, batch_labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        preds = outputs.argmax(dim=1)
        all_train_preds.extend(preds.cpu().numpy())
        all_train_labels.extend(batch_labels.cpu().numpy())
    
    train_accuracy = accuracy_score(all_train_labels, all_train_preds)
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}, Train Accuracy = {train_accuracy:.4f}")

In [ ]:
model.eval()
all_test_preds = []
all_test_labels = []
with torch.no_grad():
    for batch_data, batch_labels in tqdm(test_loader, desc="Evaluating"):
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)
        outputs = model(batch_data)
        preds = outputs.argmax(dim=1)
        all_test_preds.extend(preds.cpu().numpy())
        all_test_labels.extend(batch_labels.cpu().numpy())
    
    test_accuracy = accuracy_score(all_test_labels, all_test_preds)
    print(f"Epoch {epoch+1}: Test Accuracy = {test_accuracy:.4f}")

In [18]:
class SiameseDataset(Dataset): 
    def __init__(self, base_dataset, similar_prob=0.5, min_time_gap=5):
        self.base_dataset = base_dataset
        self.similar_prob = similar_prob
        self.min_time_gap = min_time_gap

        self.subject_file_to_indices = {}
        for i, entry in enumerate(base_dataset.indices):
            subject = entry[-1]
            file_path = entry[3]
            key = (subject, file_path)
            if key not in self.subject_file_to_indices:
                self.subject_file_to_indices[key] = []
            self.subject_file_to_indices[key].append(i)
            
        self.all_indices = list(range(len(base_dataset)))

    def __len__(self):
        return len(self.base_dataset)
    
    def __getitem__(self, idx):
        if random.random() < self.similar_prob:
            label = 1

            index1 = idx
            subject, recording, aug_idx, file_path, label = self.base_dataset.indices[index1]
            # Only choose candidates that come from the same subject and same file.
            key = (subject, file_path)
            candidate_indices = self.subject_file_to_indices.get(key, [])

            if len(candidate_indices) < 2:
                label = 0
                index2 = random.choice(self.all_indices)
                
                while self.base_dataset.indices[index2][-1] == label:
                    index2 = random.choice(self.all_indices)

            else:
                index2 = random.choice(candidate_indices)

                while index2 == index1 or (
                    abs(self.base_dataset.indices[index1][2] - self.base_dataset.indices[index2][2]) < self.min_time_gap
                ):
                    index2 = random.choice(candidate_indices)
        
        else: 
            label = 0
            index1 = idx
            subject1 = self.base_dataset.indices[index1][-1]
            index2 = random.choice(self.all_indices)
            
            while self.base_dataset.indices[index2][-1] == subject1:
                index2 = random.choice(self.all_indices)

        sample1, _ = self.base_dataset[index1]
        sample2, _ = self.base_dataset[index2]
        return sample1, sample2, torch.tensor(label, dtype=torch.float32)

In [19]:
siamese_dataset = SiameseDataset(dataset, similar_prob=0.5, min_time_gap=5)
siamese_loader = DataLoader(siamese_dataset, batch_size=32, shuffle=True, num_workers=0)

In [20]:
print("Dataset length:", len(siamese_dataset))

# Retrieve and inspect the first sample.
sample1, sample2, pair_label = siamese_dataset[0]
print("Pair label:", pair_label.item())
print("Sample 1 shape:", sample1.shape)
print("Sample 2 shape:", sample2.shape)

Dataset length: 26715
Pair label: 0.0
Sample 1 shape: torch.Size([30, 160, 3])
Sample 2 shape: torch.Size([30, 160, 3])
